In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import pathlib
import scipy.io as sio
from os.path import dirname, join as pjoin

In [5]:
print(pathlib.Path().absolute())
#pathCourant = 'D:\\Programmation\\ipssi_cours\\Master 2\\Python ML\\DeepLearning_classificationImages'

D:\Programmation\ipssi_cours\Master 2\Python ML\DeepLearning_classificationImages


In [6]:
train_data = sio.loadmat('train_data.mat')
test_data = sio.loadmat('train_data.mat')

In [7]:
training_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [9]:
dataset = training_data.flow_from_directory(
    pathlib.Path().joinpath('Images').absolute(),
    target_size=(150,150),
    class_mode='categorical'
)

Found 20580 images belonging to 120 classes.
